# 获取河流冲积物区域

In [1]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from rasterio import features
from scipy.ndimage import generic_filter, distance_transform_edt
from scipy.interpolate import griddata
from shapely.geometry import shape
import pygeos

In [2]:
# 地形分析函数
def calculate_slope(dem, cell_size):
    """计算DEM的坡度
    Args:
        dem: 数字高程模型数组
        cell_size: 像元大小
    Returns:
        坡度数组（度）
    """
    dy, dx = np.gradient(dem, cell_size)
    slope_radians = np.arctan(np.sqrt(dx*dx + dy*dy))
    return np.degrees(slope_radians)

def calculate_elevation_difference(dem, river_raster, cell_size, max_distance=1000):
    """计算与最近河流点的高程差
    Args:
        dem: 数字高程模型数组
        river_raster: 河流栅格掩膜
        cell_size: 像元大小
        max_distance: 最大计算距离（米）
    Returns:
        高程差数组
    """
    distance = distance_transform_edt(~river_raster) * cell_size
    river_coords = np.column_stack(np.where(river_raster))
    river_elevations = dem[river_raster]
    
    y, x = np.mgrid[0:dem.shape[0], 0:dem.shape[1]]
    positions = np.column_stack((y.ravel(), x.ravel()))
    elevation_diff = griddata(river_coords, river_elevations, positions, method='nearest').reshape(dem.shape)
    elevation_diff = dem - elevation_diff
    elevation_diff[distance > max_distance] = np.nan
    
    return elevation_diff

def local_relief(dem, size=5):
    """计算局部地形起伏度
    Args:
        dem: 数字高程模型数组
        size: 计算窗口大小
    Returns:
        局部起伏度数组
    """
    return generic_filter(dem, np.ptp, size=size)

# 主要处理函数
def identify_alluvial_areas(river_vector_path, dem_path, save_path, 
                          buffer_distance=300, slope_threshold=5, 
                          elevation_threshold=5, relief_threshold=5):
    """识别潜在的河流冲积区
    Args:
        river_vector_path: 河流矢量文件路径
        dem_path: DEM文件路径
        save_path: 结果保存路径
        buffer_distance: 缓冲区距离（米）
        slope_threshold: 坡度阈值（度）
        elevation_threshold: 高程差阈值（米）
        relief_threshold: 地形起伏阈值（米）
    """
    # 读取数据
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        transform = src.transform
        crs = src.crs
    
    rivers = gpd.read_file(river_vector_path)
    river_raster = geometry_mask(rivers.geometry, out_shape=dem.shape, transform=transform, invert=True)
    cell_size = transform[0]
    
    # 计算地形指标
    slope = calculate_slope(dem, cell_size)
    elevation_diff = calculate_elevation_difference(dem, river_raster, cell_size, max_distance=buffer_distance)
    relief = local_relief(dem)
    
    # 识别冲积区
    potential_alluvium = (
        (slope < slope_threshold) &
        (elevation_diff < elevation_threshold) &
        (relief < relief_threshold) &
        (~np.isnan(elevation_diff))
    )
    
    # 保存结果
    save_name = f"potential_alluvium_{buffer_distance}_{slope_threshold}_{elevation_threshold}_{relief_threshold}.tif"
    with rasterio.open(os.path.join(save_path, save_name), 'w', 
                      driver='GTiff', height=dem.shape[0], width=dem.shape[1],
                      count=1, dtype=rasterio.uint8, crs=crs, transform=transform) as dst:
        dst.write(potential_alluvium.astype(rasterio.uint8), 1)
    
    print(f"分析完成，结果已保存为 '{save_name}'")

def enhanced_alluvial_processing(input_raster_path, output_vector_path, 
                               min_area_size=500, buffer_distance=100):
    """对识别的冲积区进行增强处理
    Args:
        input_raster_path: 输入栅格文件路径
        output_vector_path: 输出矢量文件路径
        min_area_size: 最小保留面积（平方米）
        buffer_distance: 缓冲区距离（米）
    """
    # 读取数据
    with rasterio.open(input_raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
    
    # 栅格转矢量
    raster_data = (raster_data == 1).astype(np.uint8)
    shapes = features.shapes(raster_data, transform=transform)
    geometries = [shape(geom) for geom, val in shapes if val == 1]
    
    # 创建GeoDataFrame并处理
    gdf = gpd.GeoDataFrame({'geometry': geometries}, crs=crs).explode(index_parts=True)
    
    # 空间处理
    pygeos_geoms = gdf.geometry.apply(lambda geom: pygeos.from_shapely(geom))
    buffered = pygeos.buffer(pygeos_geoms, buffer_distance)
    reverse_buffered = pygeos.buffer(buffered, -buffer_distance)
    union = pygeos.union_all(reverse_buffered)
    result = pygeos.get_parts(union)
    
    # 转回GeoDataFrame
    final_gdf = gpd.GeoDataFrame({
        'geometry': [pygeos.to_shapely(geom) for geom in result]
    }, crs=crs)
    
    # 面积计算和筛选
    final_gdf['area'] = final_gdf.geometry.area
    final_gdf = final_gdf[final_gdf['area'] >= min_area_size]
    
    # 几何简化和修复
    final_gdf['geometry'] = final_gdf.geometry.simplify(tolerance=1, preserve_topology=True)
    final_gdf['geometry'] = final_gdf.geometry.apply(lambda x: x.buffer(0) if not x.is_valid else x)
    final_gdf = final_gdf.dropna(subset=['geometry'])
    
    # 保存结果
    final_gdf.to_file(output_vector_path)
    
    print(f"处理完成，结果已保存为 '{output_vector_path}'")
    print(f"最终处理的多边形数量: {len(final_gdf)}")
    print(f"最小面积: {final_gdf['area'].min():.2f}, 最大面积: {final_gdf['area'].max():.2f}")
    
def process_alluvial_areas(
    river_vector_path,
    dem_path,
    output_dir,
    buffer_distance=300,
    slope_threshold=5,
    elevation_threshold=5,
    relief_threshold=5,
    min_area_size=500,
    smoothing_buffer=100
):
    """一站式处理河流冲积区识别和优化
    
    Args:
        river_vector_path: 河流矢量文件路径
        dem_path: DEM文件路径
        output_dir: 输出目录
        buffer_distance: 河流缓冲区距离（米）
        slope_threshold: 坡度阈值（度）
        elevation_threshold: 高程差阈值（米）
        relief_threshold: 地形起伏阈值（米）
        min_area_size: 最小保留面积（平方米）
        smoothing_buffer: 平滑缓冲区距离（米）
    
    Returns:
        tuple: (栅格结果路径, 矢量结果路径)
    """
    # 构建输出文件路径
    raster_output = os.path.join(
        output_dir, 
        f"potential_alluvium_{buffer_distance}_{slope_threshold}_{elevation_threshold}_{relief_threshold}.tif"
    )
    vector_output = os.path.join(
        output_dir,
        f"alluvial_areas_final_{buffer_distance}_{slope_threshold}_{elevation_threshold}_{relief_threshold}_{min_area_size}_{smoothing_buffer}.shp"
    )
    
    print("步骤1: 识别潜在冲积区...")
    identify_alluvial_areas(
        river_vector_path=river_vector_path,
        dem_path=dem_path,
        save_path=output_dir,
        buffer_distance=buffer_distance,
        slope_threshold=slope_threshold,
        elevation_threshold=elevation_threshold,
        relief_threshold=relief_threshold
    )
    
    print("步骤2: 优化处理结果...")
    enhanced_alluvial_processing(
        input_raster_path=raster_output,
        output_vector_path=vector_output,
        min_area_size=min_area_size,
        buffer_distance=smoothing_buffer
    )
    
    print(f"\n处理完成！")
    print(f"栅格结果: {raster_output}")
    print(f"矢量结果: {vector_output}")
    
    return raster_output, vector_output


In [ ]:
# 冲击区栅格生成参数
river_vector_path = r"F:\cache_data\shp_file\qz\qz_river\qz_river.shp" # 河流矢量路径
dem_path = r"F:\tif_features\county_feature\qz\dem.tif" # DEM路径
save_path = r"F:\cache_data\shp_file\qz\qz_river" # 保存路径
buffer_distance = 200 # 缓冲距离
slope_threshold = 5 # 坡度阈值
elevation_threshold = 2 # 高程差阈值
relief_threshold = 8 # 局部起伏阈值
# 冲击区面状化参数
min_area_size = 500 # 面状最小面积
smoothing_buffer = 50 # 面状平滑缓冲区距离

# 运行
process_alluvial_areas(river_vector_path, dem_path, save_path, 
                       buffer_distance, slope_threshold, elevation_threshold, relief_threshold, 
                       min_area_size, smoothing_buffer)